# Topic Modelling

In general, topic modeling will serve us to cluster texts that are similar to one another. Why might we want this? That is because when getting data from the Internet, we will usually have quite a big number of texts. It will be very ineffective to read all of them. Instead, we might want to read just a few from each group to understand what are the different types of texts (narratives). For example, it is probably relatively easy to gather texts about the climate emergency. However, the question is in what context it is described, and whether this context has changed over the years or not.

<center><img src="png/topic_modeling.png" width = 600/></center>

If everything goes according to the plan, under topic modeling we will discuss three models:

1. `tf-idf` (term frequency - inverse document frequency) -- a very simple model that will allow us to find similarities between texts.
2. `LDA` (Latent Dirichlet Allocation) -- model that explains a set of observations through unobserved groups, and each group explains why some parts of the data are similar.
3. `BERTopic` -- a new approach to topic modeling that uses the latest advances in transformers-based machine learning for Natural Language Processing.

However, before we start with anything fancy let's start with the basics and define some key concepts that will serve us in our journey with topic modeling.

1. `Document` -- some text.
2. `Corpus` -- a collection of documents (texts).
3. `Vector` -- a mathematical representation of a document (text).
4. `Model` -- an algorithm for transforming vectors from one representation to another.

## Document

A document is basically a string that might be the length of a single tweet or [A Litle Life](https://en.wikipedia.org/wiki/A_Little_Life). In _Python_ it will look more or less like what we could have expected. Below, we have the text of the last tweet by [The Last Week Tonight with John Oliver](https://www.youtube.com/watch?v=dQw4w9WgXcQ) account:

In [99]:
document = "Human machine interface for lab abc computer applications"

## Corpus

On the other hand, a corpus is a collection of documents (texts). For us, they will play two roles:

1. They are the input for training a model. This is going to be our main use case because among others it will allow for separating common themes and topics.
2. They organize documents. After training, a topic model can be used to extract topics from new documents (documents not seen in the training corpus). In other words, using the trained model we will be able to say how similar is a new document to a group of documents from our corpus.

At this point, we can think about a corpus as a list of strings. Consider the following. These are the 20 last tweets by [The Last Week Tonight with John Oliver](https://twitter.com/lastweektonight). We could have imported them to _Python_ in a more efficient way than this but at this point, this is not important.

In [100]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

**Important**: The above example loads the entire corpus into memory. In practice, corpora may be very large, so loading them into memory may be inconvenient (to say at least). However, for now, we are leaving it as it is.

Before we move any further, we need to tokenize our corpus and also remove stop words. We may just use the same code we discussed last week.

Therefore, let's start with loading relevant modules and downloading stop words for English.

In [101]:
## Import NLTK module
import nltk
## Import stop words
from nltk.corpus import stopwords
## Import function to tokenize text
from nltk.tokenize import word_tokenize
## A new type of dictionary
from collections import defaultdict

## Download stopwords list
nltk.download('stopwords')

## Assign the list of English stop words to stop_words
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mikolaj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now, we need to tokenize the texts and remove stop words. We are going to do it in one step using list comprehension because it is just the most convenient solution.

In [102]:
## Tokenize every single text. Meanwhile remove tokens
## that are shorter than 2 and are in stop_words list
texts = [ [ token for token in word_tokenize(doc.lower()) if token not in stop_words and len(token) > 1 ] 
          for doc in text_corpus ]


Let's create a mapping with the frequency of all words in the corpus. We know how to do it but let's discover a new way of doing it. Instead of using a built-in mapping, we are going to use a more fancy one. The difference is that it does not return an error when a given key does not exist. Instead, it creates a new pairing with the given key and default value. In other words, if we define new mapping as follows:

In [103]:
new_mapping = defaultdict(int)

and try to access its non-existing key, let's say `'name'`

In [104]:
new_mapping['name']

0

it returned `0` because we initialized it with a default value to `int` which is understood as `0`. What's more, when we inspect the mapping, we see that a new pairing appeared.

In [105]:
new_mapping

defaultdict(int, {'name': 0})

So it not only did not return an error. It also created a new pairing. This is pretty cool, right? Mostly because we don't have to write any more code like the following to compute the frequency of tokens.

In [106]:
## Create an empty mapping.
frequency = {}

## Compute the frequency of tokens in the whole
## corpus.
for text in texts:
    for token in text:
        if token in frequency:
            frequency[token] += 1
        else:
            frequency[token] = 1
        

In [107]:
## Create an empty fancy mapping.
frequency = defaultdict(int)

## Count the number of words all texts
for text in texts:
    for token in text:
        frequency[token] += 1

Clearly, both methods return the same results.

Ok, now let's use the frequency mapping to remove from our corpus words that appear only once. For our purposes, they are not going to be very useful because we will not be able to infer from them whether the two documents are similar. Again, we are going to use list comprehension for that.

In [108]:
## Remove tokens that appear only once in the whole corpus
processed_corpus = [ [ token for token in text if frequency[token] > 1 ] 
                     for text in texts ]

processed_corpus

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

Before proceeding, we will associate each word in the corpus with a unique integer ID. We want to do this because for a computer it will be just easier to use integers than strings (basically a string consumes more space than an integer, therefore, the computer can operate on integers faster than on string). We will use for that a new module `gensim`. It will allow for the creation of a dictionary that will store unique tokens paired with a unique integer id. In other words, we will create a kind of mapping. We can think about it a bit like a dictionary with integers as keys and tokens as values.

In [109]:
## Import gensim
from gensim import corpora

## Create a new object called Dictionary
dictionary = corpora.Dictionary(processed_corpus)

## Let's print it out
print(dictionary)

Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...>


As you see it is not a mapping per se. But we can easily convert it to an ordinary mapping by iteraiting over it.

In [110]:
{ key : value for key, value in dictionary.items() }

{0: 'computer',
 1: 'human',
 2: 'interface',
 3: 'response',
 4: 'survey',
 5: 'system',
 6: 'time',
 7: 'user',
 8: 'eps',
 9: 'trees',
 10: 'graph',
 11: 'minors'}

**NOTE**: We have here a relatively small corpus, therefore, there are only 12 different tokens. For a larger corpus, dictionaries can become massive, like over a thousand tokens. Among others, this is why it is not an ordinary dictionary but stores the data in a more efficient and memory-friendly way. The other property of this Dictionary is that we don't have to have all documents in the working memory at the same time to create it. There is an easy way to create it by processing one document at a time. However, we are not going to discuss it right now because right now we focus on the general idea.

## Vector

What happened so far was relatively easy. More or less we converted a set of texts into a new object which is called Dictionary (it is not the same as `dict` or a mapping). Now, we would like to convert the tokens we have into vectors. We need that because to infer the latent structure in our corpus we need a way to represent documents in a way that allows for mathematical manipulation. In other words, we need to transform strings that are understandable for us into something which will be understandable for a computer. We can infer the latent structure (find similarities between texts) by understanding the meaning of the texts but computers can not really do that (don't believe people who state otherwise). Instead, they can quite well and fast perform mathematical operations. Therefore, we need a way to convert a text into a mathematical object (vector) and ask the computer to find similarities between a set of vectors using math (you probably computed the distance between two vectors in high school).

In other words, we need to find a way to represent a document (text) as a vector of features. For example, a single feature may be thought of as a question-answer pair:

1. How many times does the word *sponge* appear in the document? Zero.
2. How many paragraphs does the document consist of? Two.
3. How many fonts does the document use? Five.

If we are going to apply the same questions to every single text we can skip their text and only refer to their integer ids (such as `1`, `2`, and `3`). Therefore, the representation of a document becomes a series of pairs like `(1, 0), (2, 2), (3, 5)`. This is called a **dense vector**, because it contains an explicit answer to each of the above questions.

In practice, we can omit `0` values. We do that to save memory (usually we have much more features than just `3`). Therefore, actually, our vector would look like `(2,2), (3,5)`. This is known as a **sparse vector** or **bag-of-words vector**.

Assuming the questions are the same we can compare two texts, for example, texts with the following **bag-of-words** vectors `(2, 2), (3, 5)` and `(1, 1), (2, 2), (3, 5)`. Based on the answers to our questions we can infer that the two texts must be similar. We don't really have to employ heavy maths to say that. The only difference between these two texts is the appearance of [sponge](https://www.youtube.com/watch?v=-SuSWX8h-hg). They must be similar, at least on the examined features. We don't have to do any heavy math to examine it.

Ok, so let's have something more involving than our silly questions. Usually, under the **bag-of-words** models, each document is represented by a vector containing the frequency count of each word in the dictionary. For example, let's assume we have a dictionary containing the following words `['sponge', 'square', 'pants', 'bob']`. A document consisting of the string `"sponge pants sponge"` would then be represented by the vector `(1, 2), (2, 0), (3, 1), (4, 0)`, where the entries of the vector are (in order) the occurrences of "sponge", "square", "pants", and "bob" in the document. The only issue here is that we kind of ignored the order of words. But let's don't worry about it right now. Also, as the name of this method suggests it is a **bag-of-words** approach so it is understandable that the order does not matter too much.

Ok, it is all very good but how it connects with the creation of this Dictionary object? This is somehow exactly this. Our processed corpus has `12` unique tokens in it, which means that each document will be represented by a 12-dimensional vector under the **bag-of-words model**. We can use the dictionary to turn tokenized documents into these 12-dimensional vectors. Let's see these ids again using the `token2id` method:

In [111]:
dictionary.token2id

{'computer': 0,
 'human': 1,
 'interface': 2,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'user': 7,
 'eps': 8,
 'trees': 9,
 'graph': 10,
 'minors': 11}

So now, when we want to vectorize a phrase we will just use the above dictionary as a kind of cipher. For example, suppose we want to vectorize the phrase "Human computer interaction" (note that this phrase was not in our original corpus). We can create the **bag-of-word** representation for a document using the `doc2bow()` method of the dictionary, which returns a sparse representation of the word counts:

In [112]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(word_tokenize(new_doc.lower()))
new_vec

[(0, 1), (1, 1)]

The first entry in each tuple corresponds to the ID of the token in the dictionary, the second corresponds to the count of this token.

Note that "interaction" did not occur in the original corpus and so it was not included in the vectorization. Also, note that this vector only contains entries for words that actually appeared in the document. Because any given document will only contain a few words out of the many words in the dictionary, words that do not appear in the vectorization are represented as implicitly zero as a space-saving measure.

We can convert our entire original corpus to a list of vectors:

In [113]:
bow_corpus = [ dictionary.doc2bow(text) for text in processed_corpus ]
bow_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

**NOTE**: In our specific case we can have the whole list in the working memory but with more (and longer) documents we will not want to load all the elements at the same time. Luckily, `gensim` allows you to use any iterator that returns a single document vector at a time but we are not going to talk about it right now.

**IMPORTANT**: The distinction between a document and a vector is that the former is text, and the latter is a mathematically convenient representation of the text. Moreover, depending on how the representation was obtained, two different documents may have the same vector representations.


## Model

Before, we talk about more involving models. Let's start with the simplest transformation. So far, we have vectorized our corpus. Now, we can transform the **bag-of-words** representation into something more interesting.

One simple example of a model (transformation) is `tf-idf` (term frequency - inverse document frequency). In simple words, it returns a statistic that allows saying how important a given term is to a document in the corpus, for example how important Sirius Black is to the Harry Potter books in the collection of Harry Potter books. Therefore, this model allows for transforming vectors from the **bag-of-words** representation to a vector space where the frequency counts are weighted according to the relative rarity of each word in the corpus. Why it is useful? That is because this statistic lays the foundations for the relevance of the search.

Imagine that you have a set of Harry Potter books and you would like to order them by the relevance to the following query: "Sirius Black". A simple way to start out is by eliminating books that do not contain all two words "Sirius Black", but this still leaves many books (probably 7). To further distinguish them, we might count the number of times each term occurs in each document; the number of times a term occurs in a document is called its term frequency. However, the problem often arises when the texts differ in terms of length (and if you read Harry Potter books you probably noticed that each book was longer than the previous one). Therefore, the term frequency is simply the number of occurence of a given word ($i$) in a given document ($j$):

$$frequency_{i,j}$$

Because the term "Black" is so common, term frequency will tend to incorrectly emphasize documents that happen to use the word "Black" more frequently, without giving enough weight to the more meaningful term "Sirius". The term "Black" is not a good keyword to distinguish relevant and non-relevant documents and terms, unlike the less-common word "Sirius". Hence, an inverse document frequency factor is incorporated which diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely. In other words, "Black" is not a very specific word for our corpus. It appears in all texts very often. On the other hand, "Sirius" is more unique because it appears less often in texts, therefore, we can think about it as more important for our corpus. In other words, if the term "Sirius" appears in our text than the probability that it is a text we have been looking for is higher.

In our case, we will compute tf-idf model simply according to the following equation:

$$tf{-}idf_{i,j} = frequency_{i,j} \times log_2(\frac{D}{document\_freq_i})$$

where, $frequency_{i,j}$ is frequency of a term $i$ in document $j$; $D$ is a number of documents in the corpus; and $document\_freq_i$ is the number of documents with the term $i$.

In [114]:
## Import models from gensim
from gensim import models

## Train the model. In other words, compute
## the tf-idf statistic for each term.
tfidf = models.TfidfModel(bow_corpus)

Ok, this is pretty cool but how exactly check what are the tf-idf scores for each term? Let's start by reminding ourselves of all the tokens we have and their frequency in the whole corpus.

In [115]:
## We simply look into frequency mapping. 
## The only thing is that we remove the elements
## that appeared only once. 
{ key : value for key, value in frequency.items() if value > 1 }

{'human': 2,
 'interface': 2,
 'computer': 2,
 'survey': 2,
 'user': 3,
 'system': 4,
 'response': 2,
 'time': 2,
 'eps': 2,
 'trees': 3,
 'graph': 3,
 'minors': 2}

Now, let's examine the documents' frequency. In other words, in how many documents a given term appeared.

In [116]:
## We can extract it form our model
tfidf.dfs

{0: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 3, 6: 2, 7: 3, 8: 2, 9: 3, 10: 3, 11: 2}

This does not help much because the keys here are these unique ids. Let's do something more useful and convert the ids back to words. Unlike computers, it matters to us whether we have a number or a word.

In [117]:
## We iterate over dictionary and take id and token and convert
## tfidf.dfs into human-readable format.
{ token : tfidf.dfs[id] for id, token in dictionary.items() }

{'computer': 2,
 'human': 2,
 'interface': 2,
 'response': 2,
 'survey': 2,
 'system': 3,
 'time': 2,
 'user': 3,
 'eps': 2,
 'trees': 3,
 'graph': 3,
 'minors': 2}

What we can see is that for example word "system" appears 4 times in the whole corpus but only in 3 documents. Other words appear in every document they appear only once.

So far so good. Let's now see idf for each word. In other words:

$$idf_i = log_2(\frac{D}{document\_freq_i})$$

In [92]:
## We iterate over dictionary and take id and token and convert
## tfidf.dfs into human-readable format.
{ token : tfidf.idfs[id] for id, token in dictionary.items() }

{'computer': 2.1699250014423126,
 'human': 2.1699250014423126,
 'interface': 2.1699250014423126,
 'response': 2.1699250014423126,
 'survey': 2.1699250014423126,
 'system': 1.5849625007211563,
 'time': 2.1699250014423126,
 'user': 1.5849625007211563,
 'eps': 2.1699250014423126,
 'trees': 1.5849625007211563,
 'graph': 1.5849625007211563,
 'minors': 2.1699250014423126}

In [121]:
print(dir(dictionary))

['__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_adapt_by_suffix', '_load_specials', '_save_specials', '_smart_save', 'add_documents', 'add_lifecycle_event', 'cfs', 'compactify', 'dfs', 'doc2bow', 'doc2idx', 'filter_extremes', 'filter_n_most_frequent', 'filter_tokens', 'from_corpus', 'from_documents', 'get', 'id2token', 'items', 'iteritems', 'iterkeys', 'itervalues', 'keys', 'lifecycle_events', 'load', 'load_from_text', 'merge_with', 'most_common', 'num_docs', 'num_nnz', 'num_pos', 'patch_with_special_tokens', 'save', 'save_as_text', 'token2id', 'values']


In [122]:
dictionary.cfs

{1: 2, 2: 2, 0: 2, 4: 2, 7: 3, 5: 4, 3: 2, 6: 2, 8: 2, 9: 3, 10: 3, 11: 2}

In [118]:
## We iterate over dictionary and take id and token and convert
## tfidf.dfs into human-readable format.
{ token : tfidf[(] for id, token in dictionary.items() }

TypeError: 'int' object is not iterable

Finally, let's see tf-idf statistics for all the words.

In [94]:
bow_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [95]:
{ key : value for key, value in frequency.items() if value > 1 }

{'human': 2,
 'interface': 2,
 'computer': 2,
 'survey': 2,
 'user': 3,
 'system': 4,
 'response': 2,
 'time': 2,
 'eps': 2,
 'trees': 3,
 'graph': 3,
 'minors': 2}

In [96]:
words = word_tokenize("system minors".lower())

words_bow = dictionary.doc2bow(words)

words_bow

[(5, 1), (11, 1)]

In [97]:
help(tfidf.dfs)

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Built-in subclasses:
 |      StgDict
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>va

In [98]:
bow_words

[(5, 1), (11, 1)]